# Stochastic Variational Method with RL algorithms

In [1]:
import numpy as np
import gym
import torch
import subprocess
import os
import h5py
import pickle

## Expoloring environment

In [2]:
env = gym.make('svm_env:svmEnv-v1', file_sigmas ="./svmCodeSVD/sigmas.dat" )

print('### Env Name ######', env.unwrapped.spec.id)

obs_space = env.observation_space

print('###### Observation space ####### \n', obs_space)

state_size = env.observation_space.shape[-1]

print('###### Size of observation space ####### \n', state_size)

act_space = env.action_space

print('###### Action space ####### \n', act_space)

act_size = env.action_space.shape[-1]

print('###### Number of actions ####### \n', act_size)

state = env.reset()

print('##### State after reset ###### \n', state)

print('##### File where will be stored sigmas \n', env.file_sigmas)



### Env Name ###### svmEnv-v1
###### Observation space ####### 
 Box(-inf, inf, (1,), float32)
###### Size of observation space ####### 
 1
###### Action space ####### 
 Box(-1.0, 1.0, (3,), float32)
###### Number of actions ####### 
 3
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
##### State after reset ###### 
 [0.]
##### File where will be stored sigmas 
 ./svmCodeSVD/sigmas.dat


## Twin Delayed DDPG (TD3) from `stable_baseline3`

In [ ]:
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise, OrnsteinUhlenbeckActionNoise

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(act_size), sigma=0.2 * np.ones(act_size))

model = DDPG("MlpPolicy", env, action_noise=action_noise, batch_size=64, gamma=1.0, verbose=1, seed=0
            , tensorboard_log=logdir )

# (policy, env, learning_rate=0.001, buffer_size=1000000,learning_starts=100, batch_size=100, 
# tau=0.005, gamma=0.99, train_freq=(1, 'episode'),  gradient_steps=- 1, action_noise=None, 
# replay_buffer_class=None, replay_buffer_kwargs=None,  optimize_memory_usage=False, 
# tensorboard_log=None, create_eval_env=False, policy_kwargs=None,  verbose=0, seed=None, 
# device='auto', _init_setup_model=True)

model.learn(total_timesteps=300, log_interval = 5, n_eval_episodes = 1)

# learn(total_timesteps, callback=None, log_interval=4, eval_env=None, eval_freq=- 1,
# n_eval_episodes=5, tb_log_name='DDPG', eval_log_path=None, reset_num_timesteps=True)

## PPO with GAE from `stable_baseline3` 

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

total_t_steps = 20

model = PPO("MlpPolicy", env, verbose=1, gamma = 1.0, tensorboard_log=logdir, batch_size=2, n_steps=2)

# classstable_baselines3.ppo.PPO(policy, env, learning_rate=0.0003, n_steps=2048, 
#         batch_size=64, n_epochs=10, gamma=0.99, gae_lambda=0.95, clip_range=0.2, 
#         clip_range_vf=None, ent_coef=0.0, vf_coef=0.5, max_grad_norm=0.5, 
#         use_sde=False, sde_sample_freq=- 1, target_kl=None, tensorboard_log=None, 
#         create_eval_env=False, policy_kwargs=None, verbose=0, seed=None, device='auto', 
#         _init_setup_model=True)

for i in range(1,10):
    model.learn(total_timesteps=total_t_steps, reset_num_timesteps=False)

# learn(total_timesteps, callback=None, log_interval=1, eval_env=None, eval_freq=- 1, 
#       n_eval_episodes=5, tb_log_name='PPO', eval_log_path=None, reset_num_timesteps=True)

    model.save(f"{models_dir_ppo}/{total_t_steps*i}")

## From my `ddpg_agent.py` code

In [3]:
from ddpg_agent import DDPG_agent
agent = DDPG_agent(state_size, act_size, seed = 0)

In [4]:
## Save all rewards, energies and princip dims in files during training
def create_info_h5(agent, env):
    # Check if file exist and creat it
    i = 0
    while os.path.exists(f'run_{i}.hdf5'):
        i += 1
    dataFile = h5py.File(f'run_{i}.hdf5', 'a')
    
    # Create dataset to store info in hdf5 file
    info = {'alg':agent.name, 'env':env.unwrapped.spec.id}
    st = h5py.string_dtype(encoding='utf-8')
    dataFile.create_dataset('info', dtype=st)
    for k in info.keys():
        dataFile['info'].attrs[k] = info[k]

    # Create dataset to store hyperparams of the model in hdf5 file
    hyperparams = {'batch_size':agent.batch_size, 'bootstrap_size':agent.bootstrap_size \
                   , 'gamma':agent.gamma, 'tau':agent.tau,'lr_critic':agent.lr_critic \
                  , 'lr_actor':agent.lr_actor, 'update_every':agent.update_every \
                   , 'transfer_every':agent.transfer_every, 'num_update':agent.num_update \
                  , 'add_noise_every':agent.add_noise_every}
    dataFile.create_dataset('hyperparams', dtype='f')
    for k in hyperparams.keys():
        dataFile['hyperparams'].attrs[k] = hyperparams[k]
    
    # Create group for rewards, energies, princip dims, actor and critic model
    dataFile.create_group('sigmas')
    dataFile.create_group('rewards')
    dataFile.create_group('energies')
    dataFile.create_group('princip_dims')
    dataFile.create_group('actor_models')
    dataFile.create_group('critic_models')
    
    # Close and return data file name
    dataFile_name = dataFile.filename
    dataFile.close()
    
    return dataFile_name

def save_all(dat_file_name, i_ep, sigmas_i_ep, rew_i_ep, en_i_ep, pri_dim_i_ep, act_model_i_ep, cr_model_i_ep):
    # Open data file
    dat_file = h5py.File(dat_file_name, 'a')
    
    # Create datasets for rewards, energies, pri dim and store data in it 
    dat_file['sigmas'].create_dataset(f'sigmas_ep_{i_ep}', dtype='f', data=sigmas_i_ep)
    dat_file['rewards'].create_dataset(f'rew_ep_{i_ep}', dtype='f', data=rew_i_ep)
    dat_file['energies'].create_dataset(f'en_ep_{i_ep}', dtype='f', data=en_i_ep)
    dat_file['princip_dims'].create_dataset(f'pri_dim_ep_{i_ep}', dtype='i', data=pri_dim_i_ep)
    
    # Store in actor models group the network params at each ep
    actor_model = torch.load(act_model_i_ep)
    dat_file['actor_models'].create_dataset(f'act_mod_{i_ep}', dtype='f')
    for k in actor_model.keys():
        dat_file['actor_models'][f'act_mod_{i_ep}'].attrs.create(name=k,data=actor_model[k].cpu().data.numpy())
    
    # Store in actor models group the network params at each ep
    critic_model = torch.load(cr_model_i_ep)
    dat_file['critic_models'].create_dataset(f'cri_mod_{i_ep}', dtype='f')
    for k in critic_model.keys():
        dat_file['critic_models'][f'cri_mod_{i_ep}'].attrs.create(name=k,data=critic_model[k].cpu().data.numpy())
    
    # Close data file
    dat_file.close()
    
def rm_useless_file(actor_model_file, critic_model_file, file_sigmas):
    os.remove(actor_model_file)
    os.remove(critic_model_file)
    os.remove(file_sigmas)

In [5]:
## Run ddpg algs   
def run_ddpg(max_t_step = 10, n_episodes=10):
    
    # Create h5 file and store info about alg and its hypereparams
    dat_file_name = create_info_h5(agent, env)
    
    for i_ep in range(n_episodes):
        state = env.reset()
        agent.reset()
        rew_i_ep = []
        en_i_ep = []
        pri_dim_i_ep = []

        ## Training loop of each episode
        for t_step in range(max_t_step):
            action = agent.act(state)
            next_state, reward, done, info = env.step(action)
            agent.step(state, action, reward, next_state, done)
            state = next_state

            # Save rew, energies, princip dims, act and crit models
            rew_i_ep.append(reward)
            en_i_ep.append(state[0])
            pri_dim_i_ep.append(env.princp_dim)
            torch.save(agent.actor_local.state_dict(), 'checkpoint_actor.pth')
            torch.save(agent.critic_local.state_dict(), 'checkpoint_critic.pth')
            if done:
                break
                
        ## Save data during training (to not lose the work done)
        save_all(dat_file_name=dat_file_name, i_ep=int(i_ep), sigmas_i_ep=env.actions_taken \
                 , rew_i_ep=rew_i_ep, en_i_ep=en_i_ep, pri_dim_i_ep=pri_dim_i_ep \
                 , act_model_i_ep='checkpoint_actor.pth', cr_model_i_ep='checkpoint_critic.pth')
        
        print('Episode {} ... Score: {:.3f}'.format(i_ep, np.sum(rew_i_ep)))

    rm_useless_file('checkpoint_actor.pth', 'checkpoint_critic.pth', env.file_sigmas)
    return dat_file_name

In [6]:
all_data = run_ddpg(10, 10)

#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [41.860283 56.667294 43.198383]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0280577
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.4116538057774957
#### CALL STEP #### 2
Action chosen at step:  [38.077927 54.73852  48.969006]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0211214
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.9293072243940301
#### CALL STEP #### 3
Action chosen at step:  [33.459167 58.16969  63.608486]
Basis size (it should be the same of full dim) =   3
With this action the energy is:  0.0183044
With this action the full dim is:  3  and princip dim is:  3
##

With this action the energy is:  0.00952514
With this action the full dim is:  5  and princip dim is:  5
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -1.1229095242031661
#### CALL STEP #### 6
Action chosen at step:  [51.409264 26.134151 83.40143 ]
Basis size (it should be the same of full dim) =   6
With this action the energy is:  0.00481656
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.7954773966754409
#### CALL STEP #### 7
Action chosen at step:  [68.41554  33.265205 76.67396 ]
Basis size (it should be the same of full dim) =   7
With this action the energy is:  0.00086835
With this action the full dim is:  7  and princip dim is:  7
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.5209209910868662
#### CALL STEP #### 8
Action chosen at step:  [66.44039  40.85726  74.079796]
Basis size (it should be the same of full dim) =   8
With this action

With this action the energy is:  -0.0190401
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.8635019430629995
Episode 4 ... Score: -5.937
#### CALL RESET ####
Action chosen at reset:  [0.]
Actions taken at reset:  []
Energies got at reset:  [0.0]
#### CALL STEP #### 1
Action chosen at step:  [55.508633 53.50427  59.20129 ]
Basis size (it should be the same of full dim) =   1
With this action the energy is:  0.0258545
With this action the full dim is:  1  and princip dim is:  1
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.2584444595692794
#### CALL STEP #### 2
Action chosen at step:  [51.783684 66.772224 62.753433]
Basis size (it should be the same of full dim) =   2
With this action the energy is:  0.0231507
With this action the full dim is:  2  and princip dim is:  2
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -2.0704236578818342
#### CALL S

With this action the energy is:  0.00595644
With this action the full dim is:  4  and princip dim is:  4
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.8747440404462399
#### CALL STEP #### 5
Action chosen at step:  [65.6031   22.253376 54.124516]
Basis size (it should be the same of full dim) =   5
With this action the energy is:  -0.00600075
With this action the full dim is:  5  and princip dim is:  5
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  -0.04324746165746163
#### CALL STEP #### 6
Action chosen at step:  [68.93989  50.747993 52.00848 ]
Basis size (it should be the same of full dim) =   6
With this action the energy is:  -0.00685225
With this action the full dim is:  6  and princip dim is:  6
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  0.015965391599753076
#### CALL STEP #### 7
Action chosen at step:  [75.54454  49.316822 75.15082 ]
Basis size (it should be the same of full dim) =   7
With this ac

With this action the energy is:  -0.0365573
With this action the full dim is:  9  and princip dim is:  9
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  2.081638634874266
#### CALL STEP #### 10
Action chosen at step:  [36.35948  65.78604  84.630455]
Basis size (it should be the same of full dim) =   10
With this action the energy is:  -0.0365812
With this action the full dim is:  10  and princip dim is:  10
#### THE ACTION IS A GOOD ONE #### --> Store the energy got!
Reward is  2.0833006280543573
Episode 9 ... Score: 0.081


NameError: name 'dat_file' is not defined

## Random search as in original SVM

In [ ]:
state = env.reset()
scores = []
step = 0
score = 0.0

while True:
    print(".....STEP.....", step)
    action = env.action_space.sample()
    next_state, reward, done, info = env.step(action)
    step = step + 1
    score += reward
    scores.append(score)
    state = next_state
    if done:
        break